# Collections and Channels
The following sections will elaborate on the basics that are covered in this chapter. It will give examples for several of the topics covered in the Collections and Channels chapter of the book. 

## Contents
1. [Multisets](#Multisets)
    1. [Simple functorality](#Simple-functorality)
2. [Probability Distributions](#Probability-Distributions)
    1. [Biased coin distribution](#Biased-coin-distribution)
    2. [Biased dice functorality](#Biased-dice-functorality)
3. [Frequentist learning](#Frequentist-learning)
    1. [A bag of marbles](#A-bag-of-marbles)
4. [Parallel products](#Parallel-products)
    1. [Parallel dices](#Parallel-dices)
2. [References](#References)


## Multisets

### Simple functorality
Suppose we have a multiset containing several individuals of which we know their biological sex and age. The type of our multiset is then $\mathcal{M}(G \times A)$ where $G := \{F,M\}$ and $A := \{1,...,200\}$. Let's now take $\tau \in \mathcal{M}(G \times A)$ to be:

$$\begin{align*} 
\tau =\ &2|(F,14)\rangle + 1|(F,15)\rangle + 1|(F,17)\rangle + 3|(F,18)\rangle + 5|(F,20)\rangle + 2|(F,21)\rangle + 1|(F,23)\rangle\ +\\
&1|(M,14)\rangle + 1|(M,16)\rangle + 2|(M,17)\rangle + 1|(M,18)\rangle + 1|(M,20)\rangle + 1|(M,21)\rangle
\end{align*}$$

This $\tau$ can represent for example the participants of a cooking course for adolescents. Now we can use the projections to show how many individuals of each gender and each age take this class:

$$\begin{align*} 
\mathcal{M}(\pi_1)(\tau) &= 15|F\rangle + 7|M\rangle\\
\mathcal{M}(\pi_2)(\tau) &= 3|14\rangle + 1|15\rangle + 1|16\rangle + 3|17\rangle + 4|18\rangle + 6|20\rangle + 3|21\rangle + 1|23\rangle
\end{align*}$$

Since a multiset is functorial we can apply a function to it using $\mathcal{M}(f)(\tau)$. For example we can apply a function $next$ that 'ages' the class an entire year so that each participant now is one year older.

$$\begin{align*} 
next &:: (G\times A) \rightarrow (G\times A) \\
next((g,a)) &= (g,a+1)
\end{align*}$$

Let's apply the function as follows: $\mathcal{M}(next)(\tau)$. Pressing the button 'Next' below will apply $\mathcal{M}(next)(\tau)$ once. 

In [2]:
from ipywidgets import interactive_output, Button, Output
import IPython.display as d
import nbinteract as nbi

M = True
F = False
ages = [14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
    
init_ms = [(2, (F, 14)), (1, (F, 15)), (1, (F, 17)), (3, (F, 18)), (5, (F, 20)), (2, (F, 21)), (1, (F, 23)),
              (1, (M, 14)), (1, (M, 16)), (2, (M, 17)), (1, (M, 18)), (1, (M, 20)), (1, (M, 21))]
ms = init_ms

out = Output()
b = Button(description='Next', disabled=False)
reset = False

def bool_to_gender(b):
    if b == M:
        return "M"
    if b == F:
        return "F"

def heights():
    h = []
    for age in ages:
        n = 0
        for x in ms:
            if x[1][1] == age:
                n += x[0]
        h.append(n)
    return h

def next_button(button):
    global ms, reset
    if reset:
        button.description='Next'
        ms = init_ms
        reset=False
    else:  
        new_ms = []
        for x in ms:
            new_ms.append((x[0],(x[1][0], x[1][1]+1)))
            if x[1][1]+1 == 28:
                reset = True
                button.description='Reset'
        ms = new_ms
    out.clear_output()
    with out:
        disp_latex()
        d.display(nbi.bar(ages, heights()))
    
def disp_latex():
    std = ""
    for x in ms:
        std += str(x[0]) + "|(" + bool_to_gender(x[1][0]) + ", " + str(x[1][1]) + r")\rangle +"
    d.display(d.Latex(r"$\tau =" + std[:-2] + "$"))

b.on_click(next_button)
d.display(b, out)
with out:
    disp_latex()
    d.display(nbi.bar(ages, heights()))

Button(description='Next', style=ButtonStyle())

Output()

Applying $next$ to our multiset $\tau$ only affects the age of the participants. When the button's text changes to 'Reset' pressing it will reset $\tau$ back to the value we defined at the beginning of this example.

In [28]:
#Test cell to see how to import efprob as local library
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from efprob.efprob import *

rgb = State([0.3,0.2,0.5], Space("RGB", ['R','G','B']))
print( rgb )

# X and Y are spaces
def chan(f, X, Y):
    return chan_fromklmap(lambda x: point_state(f(x), Y), X, Y)

def next(x):
    if x % 2 == 0:
        return 'H'
    return 'T'

0.3|R> + 0.2|G> + 0.5|B>


## Probability Distributions
### Biased coin distribution
A multiset can also be generated using a function. Recall example 1.5.1 from the book, it defines a function $flip$ to parameterize a bias probability. We will use the following adaptation for $flip:[0,1]\rightarrow C$ with $C:=\{H,T\}$:

$$flip(r) := r|H\rangle + (1-r)|T\rangle$$

With this function a multiset can be generated that represents a biased coin toss. When $r$ changes, so do the chances of Heads or Tails. The following bar diagram shows the chances for either heads or tails, $r$ is changeable.

In [3]:
from ipywidgets import interactive_output, FloatSlider
from decimal import *
from IPython.display import *
import nbinteract as nbi

# Round down decimals to 2 positions
getcontext().prec = 2

flip_rSlider = FloatSlider(
    value=0.51,
    min=0,
    max=1,
    step=0.01,
    description='$$r:=$$'
)

flip_options = {
    'title': 'Heads or Tails',
    'xlabel': '',
    'ylabel': 'Chance between 0 and 1 with 1 being 100%',
    'ylim': (0.00, 1.01)
}

def flip(_, r):
    display(Latex(r'''$$flip(''' + str(r) + ''') := ''' +
                      str(r) + r'''|H\rangle + ''' + str(Decimal(1.0)-Decimal(r)) + r'''|T\rangle$$'''))
    return (float(Decimal(r)), (float(Decimal(1.0)-Decimal(r))))

nbi.bar(['Heads', 'Tails'], flip, r=flip_rSlider, options=flip_options)

### Biased dice functorality
Let's again look at the functorality of a multiset. Suppose we have a set that represents all sides of an ordinary $6$-sided dice, $D6 := \{1, 2, 3, 4, 5, 6\}$ and recall $C := \{ H, T \}$. Then we can define the following uniform multiset for a dice using ket-notation:

$$\begin{align*} 
\varphi &= \dfrac{1}{6}|1\rangle + \dfrac{1}{6}|2\rangle + \dfrac{1}{6}|3\rangle + \dfrac{1}{6}|4\rangle + \dfrac{1}{6}|5\rangle + \dfrac{1}{6}|6\rangle
\end{align*}$$

With $\varphi \in \mathcal{M}(D6)$. Now suppose we have the following function $f$:

$$
\begin{align*} 
f &: D6 \rightarrow C  \\
f(x) &= \left\{
\begin{array}{ll}
  H\ \text{if}\ x\ \text{is even}\\
  T\ \text{otherwise}
\end{array}
\right.
\end{align*}
$$

Now since $\mathcal{M}$ is functorial we can apply $\mathcal{M}(f)(\varphi)$ as following:

$$
\begin{align*} 
    \mathcal{M}(f)(\varphi) &= \mathcal{M}(f)(\dfrac{1}{6}|1\rangle + \dfrac{1}{6}|2\rangle + \dfrac{1}{6}|3\rangle + \dfrac{1}{6}|4\rangle + \dfrac{1}{6}|5\rangle + \dfrac{1}{6}|6\rangle)\\
    &= \dfrac{1}{6}|f(1)\rangle + \dfrac{1}{6}|f(2)\rangle + \dfrac{1}{6}|f(3)\rangle + \dfrac{1}{6}|f(4)\rangle + \dfrac{1}{6}|f(5)\rangle + \dfrac{1}{6}|f(6)\rangle\\
    &= \dfrac{1}{6}|T\rangle + \dfrac{1}{6}|H\rangle + \dfrac{1}{6}|T\rangle + \dfrac{1}{6}|H\rangle + \dfrac{1}{6}|T\rangle + \dfrac{1}{6}|H\rangle\\
    &= \dfrac{3}{6}|T\rangle + \dfrac{3}{6}|H\rangle\\
    &= \dfrac{1}{2}|T\rangle + \dfrac{1}{2}|H\rangle\\
\end{align*}
$$

In this case we used the left definition of $\mathcal{M}(f)$ namely: $\mathcal{M}(\sum_i r_i|x_i\rangle)$. Suppose we have a way to accurately increase the chance of the dice to land on $6$. This can be represented by the following function $w:[0,5] \rightarrow \mathcal{M}(D6)$:

$$
w(r) = \tfrac{1-\tfrac{r}{5}}{6}|1\rangle + \tfrac{1-\tfrac{r}{5}}{6}|2\rangle + \tfrac{1-\tfrac{r}{5}}{6}|3\rangle + \tfrac{1-\tfrac{r}{5}}{6}|4\rangle + \tfrac{1-\tfrac{r}{5}}{6}|5\rangle + \tfrac{1+r}{6}|6\rangle
$$

Since the result of $w$ is an $x\in\mathcal{M}(D6)$ we can apply $\mathcal{M}(f)(x)$. This means that dependant on how weighted the dice is we have a higher change of it being even and thus a higher change to get $H$. This can be visualized as follows:

In [9]:
from ipywidgets import interactive_output, FloatSlider
from decimal import *
import IPython.display as d
import nbinteract as nbi

# Round down decimals to 4 positions
getcontext().prec = 4

w_rSlider = FloatSlider(
    value=0.00,
    min=0,
    max=5,
    step=0.01,
    description='$$r:=$$'
)

coin_options = {
    'title': 'Chance for Heads or Tails',
    'ylabel': 'Chance between 0 and 1 with 1 being 100%',
    'ylim': (0.00, 1.01)
}

dice_options = {
    'title': 'Weighted 6-sided dice',
    'ylabel': 'Chance between 0 and 1 with 1 being 100%',
    'ylim': (0.00, 1.01)
}

def w_flip(_, r):
    even = Decimal(0)
    for i, val in enumerate(weighted_dice(r)):
        if (i+1) % 2 == 0:
            even = even + Decimal(val)
    return (float(Decimal(1)-even),float(even))

def dice(_, r):
    ret_value = []
    for val in weighted_dice(r):
        ret_value.append(float(val))
    return ret_value

def weighted_dice(r):
    ret_value = []
    rdivfive = Decimal(r)/Decimal(5)
    for pip in range(1,7):
        if pip == 6:
            ret_value.append((Decimal(1)+Decimal(r))/Decimal(6))
        else:
            ret_value.append((Decimal(1)-rdivfive)/Decimal(6))
    return ret_value

def w_disp(x):
    coin = w_flip(_, x)
    d.display(d.Latex(r"$\mathcal{M}(f)(w(" + str(x) + ")) = " + 
                          str(Decimal(coin[0]).quantize(Decimal('.001'), rounding=ROUND_HALF_DOWN)) + r"|T\rangle + " +
                          str(Decimal(coin[1]).quantize(Decimal('.001'), rounding=ROUND_HALF_DOWN)) + r"|H\rangle $"))
    
    dice = r"$w(" + str(x) + ") = "
    for i, val in enumerate(weighted_dice(x)):
        dice += str(val) + r"|" + str(i+1) + r"\rangle + "
    d.display(d.Latex(dice[:-2] + "$"))

d.display(nbi.bar(['Tails', 'Heads'], w_flip, r=w_rSlider, options=coin_options))
d.display(nbi.bar([1,2,3,4,5,6], dice, r=w_rSlider, options=dice_options))
d.display(interactive_output(w_disp, {'x': w_rSlider}))


Output()

## Frequentist learning
### A bag of marbles
Suppose we have a bag containing $12$ marbles where $1$ is red, $3$ are gold, $4$ are silver and $4$ are bronze. This bag is represented by the multiset: $\tau = 1|R\rangle + 3|G\rangle + 4|S\rangle + 4|B\rangle$. When we now randomly take a marble from this bag we have chance to get either a red, gold, silver or bronze one. We can compute this chance by computing $Flrn(\tau) = 0.083|R\rangle + 0.25|G\rangle + 0.33|S\rangle + 0.33|B\rangle$. However, every time we take a marble from the bag the chance for each color changes. This is illustrated below, pressing the button will take a marble from the bag according to the chances.

In [1]:
from ipywidgets import Output, Button
from IPython.display import *
import nbinteract as nbi
from decimal import *
from random import randrange

getcontext().prec = 2

initial_bag = [(1,"R"),(3,"G"),(4,"S"),(4,"B")]
marbles = initial_bag.copy()

def amounts_left(_):
    ret = []
    for m in marbles:
        ret.append(m[0])
    return ret

def take_marble(bag):
    if is_empty(bag):
        return "None"
    r = randrange(1, total(bag)+1)
    nSoFar = 0
    i = 0
    while r > nSoFar:
        nSoFar += bag[i][0]
        i += 1
    i -= 1
    bag[i] = (bag[i][0]-1,bag[i][1])
    return bag[i][1]

def do_button_pressed(bag, button):
    if is_empty(bag):
        reset_bag()
        button.description = 'Take a marble'
        return ""
    else:
        m = take_marble(bag)
        if is_empty(bag):
            button.description = 'Put the marbles back'
        return m
    
def reset_bag():
    global marbles
    marbles = initial_bag.copy()
    
def is_empty(bag):
    for m in bag:
        if m[0] > 0:
            return False
    return True

def total(bag):
    r = 0
    for m in bag:
        r += m[0]
    return r

def conv_latex(bag):
    r = r"$\tau = "
    for m in bag:
        r += str(m[0]) + r"|" + m[1] + r"\rangle + "
    return r[:-3] + "$"

def conv_normalised_latex(bag):
    r = r"$Flrn(\tau) = "
    t = total(bag)
    for m in bag:
        div = 0
        if not t == 0:
            div = Decimal(m[0])/Decimal(t)
        r += str(div) + r"|" + m[1] + r"\rangle + "
    return r[:-3] + "$"

bag_output = Output()

def disp_bag(b):
    m = do_button_pressed(marbles, b)
    with bag_output:
        clear_output()
        display(Latex(conv_latex(marbles)))
        display(Latex(conv_normalised_latex(marbles)))
        if not m == "":
            display("Marble taken: " + m)

bag_button = Button(description='Take a marble', disabled=False)
bag_button.on_click(disp_bag)

display(bag_button, bag_output)
with bag_output:
    display(Latex(conv_latex(marbles)))
    display(Latex(conv_normalised_latex(marbles)))

Button(description='Take a marble', style=ButtonStyle())

Output()

## Channels
### Biased dice with channels
Recall the example [Biased dice functorality](#Biased-dice-functorality) in which we used the functorality of $\mathcal{M}$ to change the fair $D6$ to a fair coin flip. This same thing can be done with channels as well. However, we will slightly change the morphism. Suppose that where the dice is rolled it is dark, and $\frac{1}{10}$ times we observe wrongly if the number is even or odd. Let's define this as a $\mathcal{D}$-channel $c$:

$$
\begin{align*} 
c &: D6\multimap\mspace{-3mu}\rightarrow C  \\
c(x) &= \left\{
\begin{array}{ll}
  \frac{1}{10}|T\rangle + \frac{9}{10}|H\rangle \ \text{if}\ x\ \text{is even}\\
  \frac{9}{10}|T\rangle + \frac{1}{10}|H\rangle\ \text{otherwise}
\end{array}
\right.
\end{align*}
$$

Now take $\omega$ to be $\frac{1}{6}|1\rangle + \frac{1}{6}|2\rangle + \frac{1}{6}|3\rangle + \frac{1}{6}|4\rangle + \frac{1}{6}|5\rangle + \frac{1}{6}|6\rangle$, then we compute $c \gg \omega$:

$$
\begin{align*} 
    c \gg \omega &= (flat \circ \mathcal{D}(c))(\omega)\\
    &= flat(\mathcal{D}(c)(\omega)) \\
    &= flat(c(\tfrac{1}{6}|1\rangle) + c(\tfrac{1}{6}|2\rangle) + c(\tfrac{1}{6}|3\rangle) + c(\tfrac{1}{6}|4\rangle) + c(\tfrac{1}{6}|5\rangle) + c(\tfrac{1}{6}|6\rangle)\\
    &= flat\left( \frac{1}{6}\left|\tfrac{9}{10}|T\rangle + \tfrac{1}{10}|H\rangle\right\rangle + \frac{1}{6}\left|\tfrac{1}{10}|T\rangle + \tfrac{9}{10}|H\rangle\right\rangle + \frac{1}{6}\left|\tfrac{9}{10}|T\rangle + \tfrac{1}{10}|H\rangle\right\rangle + \frac{1}{6}\left|\tfrac{1}{10}|T\rangle + \tfrac{9}{10}|H\rangle\right\rangle + \frac{1}{6}\left|\tfrac{9}{10}|T\rangle + \tfrac{1}{10}|H\rangle\right\rangle + \frac{1}{6}\left|\tfrac{1}{10}|T\rangle + \tfrac{9}{10}|H\rangle\right\rangle \right) \\
    &= \tfrac{3}{20}|T\rangle + \tfrac{1}{60}|H\rangle + \tfrac{1}{60}|T\rangle + \tfrac{3}{20}|H\rangle + \tfrac{3}{20}|T\rangle + \tfrac{1}{60}|H\rangle + \tfrac{1}{60}|T\rangle + \tfrac{3}{20}|H\rangle + \tfrac{3}{20}|T\rangle + \tfrac{1}{60}|H\rangle + \tfrac{1}{60}|T\rangle + \tfrac{3}{20}|H\rangle \\
    &= \tfrac{30}{60}|T\rangle + \tfrac{30}{60}|H\rangle \\
    &= \tfrac{1}{2}|T\rangle + \tfrac{1}{2}|H\rangle
\end{align*}
$$

It is visible from this calculation that even though we have an imperfect observation of whether we roll an even or an odd number in the end it doesn't matter for the final chance. However, this changes when we use a weighted dice. Recall $w$ from [Biased dice functorality](#Biased-dice-functorality), we'll use this again, but this time in combination with $c$.

In [1]:
from ipywidgets import interactive_output, FloatSlider
from decimal import *
import IPython.display as d
import nbinteract as nbi

# Round down decimals to 4 positions
getcontext().prec = 4

wc_rSlider = FloatSlider(
    value=0.00,
    min=0,
    max=5,
    step=0.01,
    description='$$r:=$$'
)

wc_coin_options = {
    'title': 'Chance for Heads or Tails',
    'ylabel': 'Chance between 0 and 1 with 1 being 100%',
    'ylim': (0.00, 1.01)
}

wc_dice_options = {
    'title': 'Weighted 6-sided dice',
    'ylabel': 'Chance between 0 and 1 with 1 being 100%',
    'ylim': (0.00, 1.01)
}

def wc_flip(_, r):
    chance = (Decimal(0),Decimal(0))
    for i, val in enumerate(wc_weighted_dice(r)):
        x = wc_c(i+1)
        chance = ((val*x[0]) + chance[0], (val*x[1]) + chance[1])
    return (float(chance[0]), float(chance[1]))

def wc_c(x):
    if (x) % 2 == 0:
        return (Decimal(1)/Decimal(10),Decimal(9)/Decimal(10))
    return (Decimal(9)/Decimal(10),Decimal(1)/Decimal(10))

def wc_dice(_, r):
    ret_value = []
    for val in wc_weighted_dice(r):
        ret_value.append(float(val))
    return ret_value

def wc_weighted_dice(r):
    ret_value = []
    rdivfive = Decimal(r)/Decimal(5)
    for pip in range(1,7):
        if pip == 6:
            ret_value.append((Decimal(1)+Decimal(r))/Decimal(6))
        else:
            ret_value.append((Decimal(1)-rdivfive)/Decimal(6))
    return ret_value

def wc_disp(x):
    coin = wc_flip(_, x)
    d.display(d.Latex(r"$c \gg w(" + str(x) + ") = " + 
                          str(Decimal(coin[0]).quantize(Decimal('.001'), rounding=ROUND_HALF_DOWN)) + r"|T\rangle + " +
                          str(Decimal(coin[1]).quantize(Decimal('.001'), rounding=ROUND_HALF_DOWN)) + r"|H\rangle $"))
    
    dice = r"$w(" + str(x) + ") = "
    for i, val in enumerate(wc_weighted_dice(x)):
        dice += str(val) + r"|" + str(i+1) + r"\rangle + "
    d.display(d.Latex(dice[:-2] + "$"))

d.display(nbi.bar(['Tails', 'Heads'], wc_flip, r=wc_rSlider, options=wc_coin_options))
d.display(nbi.bar([1,2,3,4,5,6], wc_dice, r=wc_rSlider, options=wc_dice_options))
d.display(interactive_output(wc_disp, {'x': wc_rSlider}))


Output()

When the value of $r$ is different than $0$, i.e. the result of $w$ is no longer uniform, the imperfection in the observation matters. It will now never be the case that there is $100%$ guarantee to get an even number or 'heads'. This is because of the way $flat$ functions. Take $w(5.0)$ which is $0|1\rangle + 0|2\rangle + 0|3\rangle + 0|4\rangle + 0|5\rangle + 1|6\rangle$ and let's compute $c \gg w(5.0)$:

$$
\begin{align*} 
    c \gg w(5.0) &= (flat \circ \mathcal{D}(c))(w(5.0))\\
    &= (flat \circ \mathcal{D}(c))(0|1\rangle + 0|2\rangle + 0|3\rangle + 0|4\rangle + 0|5\rangle + 1|6\rangle) \\
    &= flat\left(0\left|\tfrac{9}{10}|T\rangle + \tfrac{1}{10}|H\rangle\right\rangle + 0\left|\tfrac{1}{10}|T\rangle + \tfrac{9}{10}|H\rangle\right\rangle + 0\left|\tfrac{9}{10}|T\rangle + \tfrac{1}{10}|H\rangle\right\rangle + 0\left|\tfrac{1}{10}|T\rangle + \tfrac{9}{10}|H\rangle\right\rangle + 0\left|\tfrac{9}{10}|T\rangle + \tfrac{1}{10}|H\rangle\right\rangle + 1\left|\tfrac{1}{10}|T\rangle + \tfrac{9}{10}|H\rangle\right\rangle\right) \\
    &= 0\cdot\tfrac{9}{10}|T\rangle + 0\cdot\tfrac{1}{10}|H\rangle + 0\cdot\tfrac{1}{10}|T\rangle + 0\cdot\tfrac{9}{10}|H\rangle + 0\cdot\tfrac{9}{10}|T\rangle + 0\cdot\tfrac{1}{10}|H\rangle \\ &\ \ \ \ + 0\cdot\tfrac{1}{10}|T\rangle + 0\cdot\tfrac{9}{10}|H\rangle + 0\cdot\tfrac{9}{10}|T\rangle + 0\cdot\tfrac{1}{10}|H\rangle + 1\cdot\tfrac{1}{10}|T\rangle + 1\cdot\tfrac{9}{10}|H\rangle\\
    &= \tfrac{1}{10}|T\rangle + \tfrac{9}{10}|H\rangle
\end{align*}
$$

Observe how the inner probabilities are affected by the outer probabilities due to the $flat$ function. This means that since on a perfectly weighted dice we will only roll a $6$ our observation will always be $\tfrac{1}{10}|T\rangle + \tfrac{9}{10}|H\rangle$.

### Graphical representation of probabilistic channels
Recall that as stated in 1.7.1 that channels have a graphical representation as a transition graph.

interactive imperfect d4 to coin channel


In [44]:
from IPython.display import HTML

HTML(r'''
<script type="text/tikz">
\usetikzlibrary{arrows.meta}
\usetikzlibrary{arrows}
\begin{tikzpicture}
    \begin{scope}[every node/.style={circle,draw,fill=black}]
        \node (1) at (1,3) {};
    \end{scope}
    
    \begin{scope}[every node/.style={circle,draw}]
        \node (2) at (3,3) {2};
        \node (3) at (2,1) {3};
        \node (4) at (0,1) {4};
        \node (5) at (3,-1) {5};
    \end{scope}

    \begin{scope}[>={stealth[black]},
        every node/.style={fill=white,circle},
        every edge/.style={draw=black}]
        \path [->] (1) edge node {$2$} (2);
        \path [->] (2) edge node {$3$} (3);
        \path [->] (1) edge node {$6$} (4);
        \path [->] (4) edge node {$5$} (3);
        \path [->] (3) edge node {$3$} (5);
    \end{scope}
\end{tikzpicture}
</script>''')

## Parallel products
### Parallel dices
Suppose for some reason we want someone to roll a dice that contains $100$ possible numbers on it (i.e. the roller has a $1$ in $100$ chance for each number). Such dices exist, but in reality they take a while to land on a face since they are almost spherical. Let's say we don't have a $100$ sided dice, but we do have a $10$-sided dice, a d10. We can represent all faces on the dice as the following set $D10 := \{0,1,2,3,4,5,6,7,8,9\}$. Rolling such a dice can be represented by the following uniform probability distribution:

$$\tau = \frac{1}{10}|0\rangle + \frac{1}{10}|1\rangle + \frac{1}{10}|2\rangle + \frac{1}{10}|3\rangle + \frac{1}{10}|4\rangle + \frac{1}{10}|5\rangle + \frac{1}{10}|6\rangle + \frac{1}{10}|7\rangle + \frac{1}{10}|8\rangle + \frac{1}{10}|9\rangle$$

We now want to use this dice to create a uniform probability distribution in which every possibility has a $\frac{1}{100}$ chance. Intuitively this is easy, just roll the d10 twice and you get $100$ different combinations. In case you would like to translate this to the number sequence $[1..100]$ you can use the following:

$$ 
0, 1 = 1\\
0, 2 = 2\\
...\\
0, 9 = 9\\
0, 0 = 10\\
1, 1 = 11\\
...\\
1, 0 = 20\\
9, 0 = 100
$$

To now represent this in a multiset we can compute the parallel product of $\tau$ with itself, $\tau \otimes \tau$, as follows:

$$\begin{eqnarray*}
\tau\otimes\tau &=& \sum_{x,y} (\tau(x) \cdot \tau(y))|x,y\rangle\\
&=& (\tau(0)\cdot\tau(0))|0,0\rangle + (\tau(0)\cdot\tau(1))|0,1\rangle +\ ...\ + (\tau(9)\cdot\tau(9))|9,9\rangle\\
&=& \frac{1}{100}|0,0\rangle + \frac{1}{100}|0,1\rangle +\ ...\ + \frac{1}{100}|9,9\rangle
\end{eqnarray*}$$